## Install Dependencies

In [ ]:
!pip install openai

## Imports and Initialisation

In [ ]:
# @title
import openai
from openai.embeddings_utils import get_embedding, cosine_similarity
import pandas as pd
import time
dotenv_path = "../../.env"
load_dotenv(dotenv_path)
openai.api_key = os.environ.get("API_KEY")
last_call_time = 0

Dummy Order and Customer Data

In [ ]:
# @title
product_data = [{
    "prod_id": 1,
    "prod": "moisturizer",
    "brand":"Aveeno",
    "description": "for dry skin"
},
{
    "prod_id": 2,
    "prod": "foundation",
    "brand":"Maybelline",
    "description": "medium coverage"
},
{
    "prod_id": 3,
    "prod": "moisturizer",
    "brand":"CeraVe",
    "description": "for dry skin"
},
{
    "prod_id": 4,
    "prod": "nail polish",
    "brand":"OPI",
    "description": "raspberry red"
},
{
    "prod_id": 5,
    "prod": "concealer",
    "brand":"Chanel",
    "description": "medium coverage"
},
{
    "prod_id": 6,
    "prod": "moisturizer",
    "brand":"Ole Henkrisen",
    "description": "for oily skin"
},
{
    "prod_id": 7,
    "prod": "moisturizer",
    "brand":"CeraVe",
    "description": "for normal to dry skin"
},
{
    "prod_id": 8,
    "prod": "moisturizer",
    "brand":"First Aid Beauty",
    "description": "for dry skin"
},{
    "prod_id": 9,
    "prod": "makeup sponge",
    "brand":"Sephora",
    "description": "super-soft, exclusive, latex-free foam"
}]

customer_order_data = [
{
    "prod_id": 1,
    "prod": "moisturizer",
    "brand":"Aveeno",
    "description": "for dry skin"
},{
    "prod_id": 2,
    "prod": "foundation",
    "brand":"Maybelline",
    "description": "medium coverage"
},{
    "prod_id": 4,
    "prod": "nail polish",
    "brand":"OPI",
    "description": "raspberry red"
},{
    "prod_id": 5,
    "prod": "concealer",
    "brand":"Chanel",
    "description": "medium coverage"
},{
    "prod_id": 9,
    "prod": "makeup sponge",
    "brand":"Sephora",
    "description": "super-soft, exclusive, latex-free foam"
}]


## PreProcessing and get_embedding

In [ ]:
# @title
# Product data processing
product_data_df = pd.DataFrame(product_data)
product_data_df

,prod_id,prod,brand,description
0,1,moisturizer,Aveeno,for dry skin
1,2,foundation,Maybelline,medium coverage
2,3,moisturizer,CeraVe,for dry skin
3,4,nail polish,OPI,raspberry red
4,5,concealer,Chanel,medium coverage
5,6,moisturizer,Ole Henkrisen,for oily skin
6,7,moisturizer,CeraVe,for normal to dry skin
7,8,moisturizer,First Aid Beauty,for dry skin
8,9,makeup sponge,Sephora,"super-soft, exclusive, latex-free foam"


# Function to get the text embedding with rate limiting

In [ ]:
# @title
def get_embedding_with_rate_limit(text):
    # Calculate time elapsed since last call
    global last_call_time
    time_elapsed = time.time() - last_call_time
    engine = 'text-embedding-ada-002'
    if time_elapsed < 20:
        # Sleep for the remaining time to meet the rate limit
        sleep_time = 20 - time_elapsed
        print(f"sleeping for {sleep_time}")
        time.sleep(sleep_time)
    # Call the get_embedding function
    embedding = get_embedding(text, engine)

    # Update the last call time
    last_call_time = time.time()

    return embedding

In [ ]:
# @title
product_data_df['combined'] = product_data_df.apply(lambda row: f"{row['brand']}, {row['prod']}, {row['description']}", axis=1)

product_data_df['text_embedding'] = product_data_df.combined.apply(lambda x: get_embedding_with_rate_limit(x))
product_data_df

sleeping for 19.999992847442627
sleeping for 19.999993562698364
sleeping for 19.999993085861206
sleeping for 19.99999189376831
sleeping for 19.999993801116943
sleeping for 19.999993801116943
sleeping for 19.999993085861206
sleeping for 19.999993324279785


,prod_id,prod,brand,description,combined,text_embedding
0,1,moisturizer,Aveeno,for dry skin,"Aveeno, moisturizer, for dry skin","[-0.0054562389850616455, -0.009110474959015846..."
1,2,foundation,Maybelline,medium coverage,"Maybelline, foundation, medium coverage","[-0.01599975861608982, 0.002224505180492997, -..."
2,3,moisturizer,CeraVe,for dry skin,"CeraVe, moisturizer, for dry skin","[0.007493841927498579, -0.016976866871118546, ..."
3,4,nail polish,OPI,raspberry red,"OPI, nail polish, raspberry red","[-0.0005904638092033565, -0.013713370077311993..."
4,5,concealer,Chanel,medium coverage,"Chanel, concealer, medium coverage","[0.007154955994337797, 0.007432130165398121, 0..."
5,6,moisturizer,Ole Henkrisen,for oily skin,"Ole Henkrisen, moisturizer, for oily skin","[-0.004918951541185379, -0.02238703891634941, ..."
6,7,moisturizer,CeraVe,for normal to dry skin,"CeraVe, moisturizer, for normal to dry skin","[0.015826856717467308, -0.013203620910644531, ..."
7,8,moisturizer,First Aid Beauty,for dry skin,"First Aid Beauty, moisturizer, for dry skin","[-0.011211211793124676, -0.007694851141422987,..."
8,9,makeup sponge,Sephora,"super-soft, exclusive, latex-free foam","Sephora, makeup sponge, super-soft, exclusive,...","[0.006272361148148775, 0.0048912288621068, 0.0..."


In [ ]:
# @title
# Order data processing
customer_order_df = pd.DataFrame(customer_order_data)
customer_order_df

,prod_id,prod,brand,description
0,1,moisturizer,Aveeno,for dry skin
1,2,foundation,Maybelline,medium coverage
2,4,nail polish,OPI,raspberry red
3,5,concealer,Chanel,medium coverage
4,9,makeup sponge,Sephora,"super-soft, exclusive, latex-free foam"


In [ ]:
# @title
customer_order_df['combined'] = customer_order_df.apply(lambda row: f"{row['brand']}, {row['prod']}, {row['description']}", axis=1)

customer_order_df['text_embedding'] = customer_order_df.combined.apply(lambda x: get_embedding_with_rate_limit(x))
customer_order_df

sleeping for 19.999993562698364
sleeping for 19.999992847442627
sleeping for 19.99999189376831
sleeping for 19.999993801116943


,prod_id,prod,brand,description,combined,text_embedding
0,1,moisturizer,Aveeno,for dry skin,"Aveeno, moisturizer, for dry skin","[-0.005430120974779129, -0.009115070104598999,..."
1,2,foundation,Maybelline,medium coverage,"Maybelline, foundation, medium coverage","[-0.01599975861608982, 0.002224505180492997, -..."
2,4,nail polish,OPI,raspberry red,"OPI, nail polish, raspberry red","[-0.0006032940000295639, -0.01373579166829586,..."
3,5,concealer,Chanel,medium coverage,"Chanel, concealer, medium coverage","[0.007154955994337797, 0.007432130165398121, 0..."
4,9,makeup sponge,Sephora,"super-soft, exclusive, latex-free foam","Sephora, makeup sponge, super-soft, exclusive,...","[0.006272361148148775, 0.0048912288621068, 0.0..."


## Create Embedding for prompt

In [ ]:
# @title
prompt = "Hi! Can you recommend a good moisturizer for me?"
response = openai.Embedding.create(
    input=prompt,
    model="text-embedding-ada-002"
)
embeddings_customer_question = response['data'][0]['embedding']

# search for similarity in order data
customer_order_df['search_purchase_history'] = customer_order_df.text_embedding.apply(lambda x: cosine_similarity(x, embeddings_customer_question))
customer_order_df = customer_order_df.sort_values('search_purchase_history', ascending=False)
customer_order_df

,prod_id,prod,brand,description,combined,text_embedding,search_purchase_history
0,1,moisturizer,Aveeno,for dry skin,"Aveeno, moisturizer, for dry skin","[-0.005430120974779129, -0.009115070104598999,...",0.861103
3,5,concealer,Chanel,medium coverage,"Chanel, concealer, medium coverage","[0.007154955994337797, 0.007432130165398121, 0...",0.792031
1,2,foundation,Maybelline,medium coverage,"Maybelline, foundation, medium coverage","[-0.01599975861608982, 0.002224505180492997, -...",0.782503
4,9,makeup sponge,Sephora,"super-soft, exclusive, latex-free foam","Sephora, makeup sponge, super-soft, exclusive,...","[0.006272361148148775, 0.0048912288621068, 0.0...",0.762074
2,4,nail polish,OPI,raspberry red,"OPI, nail polish, raspberry red","[-0.0006032940000295639, -0.01373579166829586,...",0.748478


In [ ]:
# @title
# search for similarity in product data
product_data_df['search_products'] = product_data_df.text_embedding.apply(lambda x: cosine_similarity(x, embeddings_customer_question))
product_data_df = product_data_df.sort_values('search_products', ascending=False)
product_data_df

,prod_id,prod,brand,description,combined,text_embedding,search_products
0,1,moisturizer,Aveeno,for dry skin,"Aveeno, moisturizer, for dry skin","[-0.0054562389850616455, -0.009110474959015846...",0.861086
2,3,moisturizer,CeraVe,for dry skin,"CeraVe, moisturizer, for dry skin","[0.007493841927498579, -0.016976866871118546, ...",0.861061
7,8,moisturizer,First Aid Beauty,for dry skin,"First Aid Beauty, moisturizer, for dry skin","[-0.011211211793124676, -0.007694851141422987,...",0.855761
6,7,moisturizer,CeraVe,for normal to dry skin,"CeraVe, moisturizer, for normal to dry skin","[0.015826856717467308, -0.013203620910644531, ...",0.851225
5,6,moisturizer,Ole Henkrisen,for oily skin,"Ole Henkrisen, moisturizer, for oily skin","[-0.004918951541185379, -0.02238703891634941, ...",0.837511
4,5,concealer,Chanel,medium coverage,"Chanel, concealer, medium coverage","[0.007154955994337797, 0.007432130165398121, 0...",0.792031
1,2,foundation,Maybelline,medium coverage,"Maybelline, foundation, medium coverage","[-0.01599975861608982, 0.002224505180492997, -...",0.782503
8,9,makeup sponge,Sephora,"super-soft, exclusive, latex-free foam","Sephora, makeup sponge, super-soft, exclusive,...","[0.006272361148148775, 0.0048912288621068, 0.0...",0.762074
3,4,nail polish,OPI,raspberry red,"OPI, nail polish, raspberry red","[-0.0005904638092033565, -0.013713370077311993...",0.748579


## Display results

In [ ]:
top_3_purchases_df = customer_order_df.head(3)
top_3_purchases_df

,prod_id,prod,brand,description,combined,text_embedding,search_purchase_history
0,1,moisturizer,Aveeno,for dry skin,"Aveeno, moisturizer, for dry skin","[-0.005430120974779129, -0.009115070104598999,...",0.861103
3,5,concealer,Chanel,medium coverage,"Chanel, concealer, medium coverage","[0.007154955994337797, 0.007432130165398121, 0...",0.792031
1,2,foundation,Maybelline,medium coverage,"Maybelline, foundation, medium coverage","[-0.01599975861608982, 0.002224505180492997, -...",0.782503


In [ ]:
top_3_products_df = product_data_df.head(3)
top_3_products_df

,prod_id,prod,brand,description,combined,text_embedding,search_products
0,1,moisturizer,Aveeno,for dry skin,"Aveeno, moisturizer, for dry skin","[-0.0054562389850616455, -0.009110474959015846...",0.861086
2,3,moisturizer,CeraVe,for dry skin,"CeraVe, moisturizer, for dry skin","[0.007493841927498579, -0.016976866871118546, ...",0.861061
7,8,moisturizer,First Aid Beauty,for dry skin,"First Aid Beauty, moisturizer, for dry skin","[-0.011211211793124676, -0.007694851141422987,...",0.855761


## Use continued messages to refine output

In [ ]:
# @title
message_objects = []

message_objects.append({"role":"system", "content":"You're a chatbot helping customers with beauty-related questions and helping them with product recommendations"})
customer_input = "which brand makes a nail polish"
message_objects.append({"role":"user", "content": customer_input})

prev_purchases = ". ".join([f"{row['combined']}" for index, row in top_3_purchases_df.iterrows()])
prev_purchases

'Aveeno, moisturizer, for dry skin. Chanel, concealer, medium coverage. Maybelline, foundation, medium coverage'

In [ ]:
# @title
message_objects.append({"role":"user", "content": f"Here're my latest product orders: {prev_purchases}"})

message_objects.append({"role":"user", "content": f"Please give me a detailed explanation of your recommendations"})
message_objects.append({"role":"user", "content": "Please be friendly and talk to me like a person, don't just give me a list of recommendations"})

message_objects.append({"role": "assistant", "content": f"I found these 3 products I would recommend"})

products_list = []

for index, row in top_3_products_df.iterrows():
    brand_dict = {'role': "assistant", "content": f"{row['combined']}"}
    products_list.append(brand_dict)
products_list

message_objects.extend(products_list)

message_objects.append({"role": "assistant", "content":"Here's my summarized recommendation of products, and why it would suit you:"})

## Completions API to get final output

In [ ]:
completion = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=message_objects
)

print(completion.choices[0].message['content'])

For moisturizer for dry skin, I recommend Aveeno. Aveeno is known for its gentle and effective formulas that help soothe and hydrate dry skin. Their moisturizers are formulated with ingredients like oat extract and shea butter that can provide long-lasting moisture and relieve dryness. Aveeno moisturizers are also non-greasy and lightweight, making them comfortable to use throughout the day.

For concealer with medium coverage, Chanel is a great option. Chanel is a luxury brand known for their high-quality makeup products. Their concealers are often praised for their smooth texture and blendability, which can help cover imperfections and dark circles without feeling heavy or cakey. Chanel concealers also tend to have a natural finish, giving you a seamless look.

Lastly, for foundation with medium coverage, Maybelline is a popular choice. Maybelline offers a range of affordable and accessible foundations that provide buildable coverage. Their medium coverage foundations can help even o